In [7]:
import dwave_networkx as dnx
import tsplib95 as tsp
import networkx as nx
import dimod
import numpy as np

In [21]:
prob = tsp.load('burma14.tsp')
G = prob.get_graph()
qubo = dnx.traveling_salesman_qubo(G=nx.convert_node_labels_to_integers(G))
bqm = dimod.BQM.from_qubo(qubo)
mapping = bqm.relabel_variables_as_integers()
bqm.change_vartype('SPIN')
h, J, offset = bqm.to_ising()
with open('burma14.ising', 'w') as burmafile:
    burmafile.write(f'{bqm.num_variables} {len(h)} {len(J)}\n')
    for u, bias in h.items():
        burmafile.write(f'{u} {bias}\n')
    for (u, v), bias in J.items():
        burmafile.write(f'{u} {v} {bias}\n')
        
    

In [ ]:
vals = np.array([ 1.0000,  0.9933,  0.9658,  0.9681,  0.9822,  0.9850,  0.9748,  0.9945,
          0.9708,  0.9546, -0.8889,  1.0000,  0.9803,  0.9937,  0.9968,  1.0000,
          1.0000,  0.9841,  0.9978,  0.9826,  1.0000,  0.9831,  0.9883,  0.9864,
          0.9861,  0.9948,  1.0000, -1.0000,  0.9800,  1.0000,  0.9945,  0.9662,
          0.9797, -0.9454,  0.9401,  0.9974,  0.9881,  0.9999,  0.9855,  0.9723,
          1.0000,  0.9940,  0.9899,  0.9758,  1.0000,  0.9324,  1.0000,  1.0000,
          0.9505,  0.9515, -0.9780,  1.0000,  0.9406,  1.0000,  0.9956,  1.0000,
          0.9845,  1.0000,  1.0000,  0.9486, -0.9364,  0.9660,  0.9964,  0.9381,
          0.9628,  0.9191,  0.9833,  0.9809,  0.9533,  0.9537, -1.0000,  1.0000,
          0.9593,  0.9818,  0.9880,  0.9548,  0.9080,  0.9657,  0.9953,  0.9657,
          0.9977,  1.0000,  1.0000,  0.9961,  0.9864,  0.9938,  0.9967,  0.9693,
          1.0000,  1.0000,  0.9957,  0.9946,  0.9705,  0.8880,  0.9719, -0.9586,
          0.9945,  0.9489,  0.9938,  0.9760,  0.9791,  0.9932,  0.9947,  0.9615,
          0.9779,  0.9797,  1.0000,  0.9776,  0.9512,  1.0000, -0.9308,  0.9894,
          1.0000,  1.0000,  1.0000,  0.9771,  0.9906,  0.9671, -1.0000,  0.9766,
          0.9954,  1.0000,  0.9601,  0.9715,  0.9748,  0.9928,  0.9871, -0.9770,
          0.9610,  0.9583,  0.9998,  1.0000,  0.9934,  1.0000,  0.9864,  1.0000,
          0.9773,  1.0000,  0.9069,  0.9588,  0.9521,  0.9924,  0.9794,  0.9842,
          0.9644,  1.0000,  1.0000, -0.8507,  1.0000,  0.9987,  0.9728,  1.0000,
          0.9710,  0.9682,  1.0000,  1.0000,  1.0000, -0.9739,  0.9506,  0.9562,
          1.0000,  1.0000,  1.0000,  0.9893,  1.0000,  0.8275,  0.9633,  0.9750,
          1.0000,  0.9366, -1.0000,  1.0000,  1.0000,  0.9713,  0.9831,  1.0000,
          0.8955,  0.9968,  0.9920,  0.9862,  0.9897,  0.9991,  0.9966,  1.0000,
          0.9849,  1.0000,  0.9909,  1.0000,  0.9362,  0.9949,  0.9997, -1.0000,
          0.9006,  1.0000,  1.0000,  1.0000])
(vals <= 0).sum()

14

In [6]:
A

array([[7.93021412e-04, 1.21332276e-01, 4.04440920e-01, 5.59873117e-01,
        7.66058684e-01, 4.60745440e-01, 3.60824742e-01, 5.55114988e-02,
        1.26883426e-01, 2.95003965e-01, 1.24504362e-01, 4.49643140e-01,
        2.71213323e-01, 3.15622522e-01],
       [1.21332276e-01, 7.93021412e-04, 3.34655036e-01, 5.26566217e-01,
        7.90642347e-01, 4.74226804e-01, 4.02061856e-01, 1.56225218e-01,
        2.46629659e-01, 3.79857256e-01, 2.45836638e-01, 4.60745440e-01,
        3.30689929e-01, 2.98176051e-01],
       [4.04440920e-01, 3.34655036e-01, 7.93021412e-04, 2.29183188e-01,
        5.90007930e-01, 3.09278351e-01, 3.46550357e-01, 3.89373513e-01,
        5.11498810e-01, 6.97858842e-01, 4.90087232e-01, 2.96590008e-01,
        3.60824742e-01, 1.67327518e-01],
       [5.59873117e-01, 5.26566217e-01, 2.29183188e-01, 7.93021412e-04,
        3.89373513e-01, 2.10150674e-01, 3.25138779e-01, 5.26566217e-01,
        6.37589215e-01, 8.48532910e-01, 6.09040444e-01, 2.05392546e-01,
        3.957

In [27]:
A

array([[7.93021412e-04, 1.21332276e-01, 4.04440920e-01, 5.59873117e-01,
        7.66058684e-01, 4.60745440e-01, 3.60824742e-01, 5.55114988e-02,
        1.26883426e-01, 2.95003965e-01, 1.24504362e-01, 4.49643140e-01,
        2.71213323e-01, 3.15622522e-01],
       [1.21332276e-01, 7.93021412e-04, 3.34655036e-01, 5.26566217e-01,
        7.90642347e-01, 4.74226804e-01, 4.02061856e-01, 1.56225218e-01,
        2.46629659e-01, 3.79857256e-01, 2.45836638e-01, 4.60745440e-01,
        3.30689929e-01, 2.98176051e-01],
       [4.04440920e-01, 3.34655036e-01, 7.93021412e-04, 2.29183188e-01,
        5.90007930e-01, 3.09278351e-01, 3.46550357e-01, 3.89373513e-01,
        5.11498810e-01, 6.97858842e-01, 4.90087232e-01, 2.96590008e-01,
        3.60824742e-01, 1.67327518e-01],
       [5.59873117e-01, 5.26566217e-01, 2.29183188e-01, 7.93021412e-04,
        3.89373513e-01, 2.10150674e-01, 3.25138779e-01, 5.26566217e-01,
        6.37589215e-01, 8.48532910e-01, 6.09040444e-01, 2.05392546e-01,
        3.957

In [1]:
import pandas as pd
from itertools import product

In [11]:
vals = [(0, 1) for i in range(4)]
data = []
for l1, l2, l3, l4 in product(*vals):
    v1 = int(l1 or l2 or l3 or l4)
    x5 = True
    v2 = int((l1 or l2 or x5) and ((not x5) or l3 or l4))
    v3 = int((l1 or l2 or (not x5)) and ((x5) or l3 or l4))
    data.append((l1, l2, l3, l4, v1, v2, v3))

In [24]:
df = pd.DataFrame(data, 
                  columns=['$\\ell_1$', '$\\ell_2$', '$\\ell_3$', '$\\ell_4$', '$f$', '$f_{new}(x_5=1)$', '$f_{new}(x_5=0)$'])
df.loc[df['$f$'] == df['$f_{new}(x_5=1)$'], '$f_{new}(x_5=1)$'] = '\\textbf{' + df.loc[df['$f$'] == df['$f_{new}(x_5=1)$']]['$f_{new}(x_5=1)$'].astype(str) + "}"
df.loc[df['$f$'] == df['$f_{new}(x_5=0)$'], '$f_{new}(x_5=0)$'] = '\\textbf{' + df.loc[df['$f$'] == df['$f_{new}(x_5=0)$']]['$f_{new}(x_5=0)$'].astype(str) + "}"
print(df.to_latex(escape=False, index=False))

\begin{tabular}{rrrrrll}
\toprule
$\ell_1$ & $\ell_2$ & $\ell_3$ & $\ell_4$ & $f$ & $f_{new}(x_5=1)$ & $f_{new}(x_5=0)$ \\
\midrule
0 & 0 & 0 & 0 & 0 & \textbf{0} & \textbf{0} \\
0 & 0 & 0 & 1 & 1 & \textbf{1} & 0 \\
0 & 0 & 1 & 0 & 1 & \textbf{1} & 0 \\
0 & 0 & 1 & 1 & 1 & \textbf{1} & 0 \\
0 & 1 & 0 & 0 & 1 & 0 & \textbf{1} \\
0 & 1 & 0 & 1 & 1 & \textbf{1} & \textbf{1} \\
0 & 1 & 1 & 0 & 1 & \textbf{1} & \textbf{1} \\
0 & 1 & 1 & 1 & 1 & \textbf{1} & \textbf{1} \\
1 & 0 & 0 & 0 & 1 & 0 & \textbf{1} \\
1 & 0 & 0 & 1 & 1 & \textbf{1} & \textbf{1} \\
1 & 0 & 1 & 0 & 1 & \textbf{1} & \textbf{1} \\
1 & 0 & 1 & 1 & 1 & \textbf{1} & \textbf{1} \\
1 & 1 & 0 & 0 & 1 & 0 & \textbf{1} \\
1 & 1 & 0 & 1 & 1 & \textbf{1} & \textbf{1} \\
1 & 1 & 1 & 0 & 1 & \textbf{1} & \textbf{1} \\
1 & 1 & 1 & 1 & 1 & \textbf{1} & \textbf{1} \\
\bottomrule
\end{tabular}



/tmp/ipykernel_8877/1433086845.py:3: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '['\\textbf{0}' '\\textbf{1}' '\\textbf{1}' '\\textbf{1}' '\\textbf{1}'
 '\\textbf{1}' '\\textbf{1}' '\\textbf{1}' '\\textbf{1}' '\\textbf{1}'
 '\\textbf{1}' '\\textbf{1}' '\\textbf{1}']' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[df['$f$'] == df['$f_{new}(x_5=1)$'], '$f_{new}(x_5=1)$'] = '\\textbf{' + df.loc[df['$f$'] == df['$f_{new}(x_5=1)$']]['$f_{new}(x_5=1)$'].astype(str) + "}"
/tmp/ipykernel_8877/1433086845.py:4: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '['\\textbf{0}' '\\textbf{1}' '\\textbf{1}' '\\textbf{1}' '\\textbf{1}'
 '\\textbf{1}' '\\textbf{1}' '\\textbf{1}' '\\textbf{1}' '\\textbf{1}'
 '\\textbf{1}' '\\textbf{1}' '\\textbf{1}']' has dtype incompatible with int64, please e